In [ ]:
# torch with cuda
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from ultralytics import YOLO
import torch
import os

# Load a model
# model = YOLO("best.onnx")
yolo_model_s = YOLO("yolo11s.pt")
yolo_model_m = YOLO("yolo11m.pt")
# path = os.path.join(os.path.abspath("."),"License Plate Recognition.v6i.yolov11.zip")

# # Train the model
# train_results = model.train(
#     data=path,  # path to dataset YAML
#     epochs=100,  # number of training epochs
#     # workers=100,
#     # patience=100,
#     imgsz=640,  # training image size
#     device="cuda" if torch.cuda.is_available() else "cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
# )

# # Evaluate model performance on the validation set
# metrics = model.val()

# # Export the model to ONNX format
# path = model.export(format="onnx")  # return path to exported model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from ultralytics import YOLO
from ultralytics.engine.results import Boxes, Results
from ultralytics.data.loaders import LoadStreams
import os
import cv2
from matplotlib import pyplot as plt
from safe_video import NumberPlateRecognition
import numpy as np
import torch
yolo_model_path = os.path.join(os.path.abspath("."), "models", "yolo11s.pt")
yolo_model = YOLO(yolo_model_path, task='detect')
plate_model_path = os.path.join(os.path.abspath("."),"models","first10ktrain","weights","best.pt")
plate_model = YOLO(plate_model_path, task='detect')
rec = NumberPlateRecognition()


In [ ]:
image = cv2.imread("example.jpg")

image = image[:, :, ::-1]
box, conf, _, _ = rec.chained_detection(image, "car").tuple()
result = rec.blur_image(image, box)
plt.imshow(result)
plt.show()

In [ ]:
display_width = 1200
display_height = 800
frame_counter = 0


cap = cv2.VideoCapture(r"c:\Users\Emre\Videos\test3.mp4")
#cap = cv2.VideoCapture("video1.mp4")
while cap.isOpened():
    success, frame = cap.read()
    if success:
        frame_counter += 1
        if frame_counter % 2 != 0:
            continue
        results = yolo_model.track(frame, persist=True,  tracker="bytetrack.yaml" ) # yolo_model(frame, imgsz=320 ,verbose = False)
        # display(results[0].boxes)
        #annotated_frames = results[0].plot() if results[0] is not None else frame
        
        #frames = results[0].plot() if results[0] is not None else Boxes()
        boxes = results[0].boxes if results[0] is not None else Boxes()

        if boxes:
            for box in boxes.data:
                x1, y1, x2, y2 = map(int, box[:4])  
                conf = box[5]
                #cls = int(box[6])
                
                if conf > 0.5: 
                    cropeed_image = frame[y1:y2, x1:x2]
                    
                    plate_results  = plate_model.track(cropeed_image, persist=True, tracker="bytetrack.yaml" )[0] #persist=True, conf=conf, iou=0.5, show=True, tracker="bytetrack.yaml"
                    plate_boxes  = plate_results.boxes.cpu().numpy()
                    if plate_boxes is not None:
                        for plate_box in plate_boxes.data:
                            # coordinates of the plate box
                            px1, py1, px2, py2 = map(int, plate_box[:4])
                            
                            
                            tx1, ty1, tx2, ty2 = px1 + x1, py1 + y1, px2 + x1, py2 + y1
                            plate_region = frame[ty1:ty2, tx1:tx2]
                            grrr = cv2.GaussianBlur(plate_region, (25, 25), 0)
                            frame[ty1:ty2, tx1:tx2] = grrr
                            
                            cv2.rectangle(frame, (tx1, ty1), (tx2, ty2), (0, 255, 0), 5)
  
                        

                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 5)
                    
        resized_frame = cv2.resize(frame, (display_width, display_height))
        
        cv2.imshow("Tracking", resized_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
                
                
    

In [ ]:
video_source = "video1.mp4"

# user can set these values in the oberlay
conf = 0.4
buffer = True

output_video = None

def on_predict_batch_end(predictor):
    global output_video

    _, image, _ = predictor.batch
    image = image if isinstance(image, list) else [image]
    
    for result in predictor.results:
        if result.boxes is  None:
            continue
        boxes = result.boxes.data.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            cropped_image = image[0][y1:y2, x1:x2]
            
            plate_results = plate_model(cropped_image)
            plate_boxes = plate_results[0].boxes.data.cpu().numpy()  
            if plate_boxes.size == 0:
                continue
            
            for plate_box in plate_boxes:
                px1, py1, px2, py2 = map(int, plate_box[:4])
                tx1, ty1, tx2, ty2 = px1 + x1, py1 + y1, px2 + x1, py2 + y1
                plate_region = image[0][ty1:ty2, tx1:tx2]
                grrr = cv2.GaussianBlur(plate_region, (25, 25), 0)
                image[0][ty1:ty2, tx1:tx2] = grrr
                cv2.rectangle(image[0], (tx1, ty1), (tx2, ty2), (0, 255, 0), 2)
                cv2.rectangle(image[0], (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.imshow("Tracking", image[0])
    if cv2.waitKey(1) & 0xFF == ord('q'):    
        return False
        
    # # Save the output video        
    # if output_video is None:
    #     h, w, _ = image[0].shape
    #     output_video = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30, (w, h))
    # output_video.write(image[0])
    
yolo_model.add_callback("on_predict_batch_end", on_predict_batch_end)
detected_cars = yolo_model.track(source=video_source, conf = conf, iou = 0.6, vid_stride= 1, stream_buffer = buffer, tracker="bytetrack.yaml", classes = 2, save = False)

cv2.destroyAllWindows()



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/825) d:\uni\Semester_5\Praktikum\NumberPlateRecognition\video1.mp4: 384x640 1 car, 38.0ms

0: 512x640 1 License_Plate, 20.0ms
Speed: 1.0ms preprocess, 20.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
video 1/1 (frame 2/825) d:\uni\Semester_5\Praktikum\NumberPlateRecognition\video1.mp4: 384x640 1 car, 17.0ms

0: 512x640 1 License_Plate, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at

In [ ]:
print(rec.get_classes())
# print(rec.result)